In [1]:
import tensorflow as tf
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
### For training ,
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)   
test_datagen=ImageDataGenerator(rescale=1./255)          

path_train = "C:/Users/justi/Neural Network Projects/Convolutional Neural Network/Traffic Signal Analysis/Training"

train_generator = train_datagen.flow_from_directory(directory=path_train,
                                                        target_size=(60, 60),
                                                        color_mode="grayscale",
                                                        batch_size=75,
                                                        class_mode="sparse",
                                                        subset='training',
                                                        seed=42)

Found 3683 images belonging to 62 classes.


In [4]:
validation_generator = train_datagen.flow_from_directory(
    directory=path_train,
    target_size=(60, 60),
    batch_size=75,
    color_mode="grayscale",
    class_mode='sparse',
    subset='validation') 

Found 892 images belonging to 62 classes.


In [5]:
path_test = "C:/Users/justi/Neural Network Projects/Convolutional Neural Network/Traffic Signal Analysis/Testing"

test_generator = test_datagen.flow_from_directory(directory=path_test,
                                                  target_size=(60, 60),
                                                  color_mode="grayscale",
                                                  batch_size=1,
                                                  class_mode=None,
                                                  shuffle=False,
                                                  seed=42
                                                  )        

Found 2520 images belonging to 62 classes.


In [6]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.initializers import he_uniform
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization

In [7]:
model=Sequential()
model.add(Conv2D(128,(3,3),activation='relu',input_shape=(60,60,1),padding='same',kernel_initializer='he_uniform')) 
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))    
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_initializer='he_uniform'))    
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(100,activation='relu',kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.2)) 
model.add(Dense(50,activation='relu',kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(62,activation='softmax'))     

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 128)       1280      
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 60, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 32)        1

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
batch_size=75
bs_model=model.fit(train_generator,steps_per_epoch=3683//batch_size,epochs=10,verbose=1,validation_data=validation_generator)

Epoch 1/10
49/49 [==============================] - 165s 3s/step - loss: 3.2112 - accuracy: 0.2813 - val_loss: 3.2511 - val_accuracy: 0.3397
Epoch 2/10
49/49 [==============================] - 128s 3s/step - loss: 1.7897 - accuracy: 0.6353 - val_loss: 2.4755 - val_accuracy: 0.5482
Epoch 3/10
49/49 [==============================] - 142s 3s/step - loss: 1.2050 - accuracy: 0.7589 - val_loss: 2.0302 - val_accuracy: 0.5852
Epoch 4/10
49/49 [==============================] - 130s 3s/step - loss: 0.8741 - accuracy: 0.8240 - val_loss: 1.4527 - val_accuracy: 0.7007
Epoch 5/10
49/49 [==============================] - 149s 3s/step - loss: 0.6551 - accuracy: 0.8661 - val_loss: 1.1328 - val_accuracy: 0.7623
Epoch 6/10
49/49 [==============================] - 166s 3s/step - loss: 0.4958 - accuracy: 0.9052 - val_loss: 0.8844 - val_accuracy: 0.8105
Epoch 7/10
49/49 [==============================] - 139s 3s/step - loss: 0.3651 - accuracy: 0.9393 - val_loss: 0.7332 - val_accuracy: 0.8419
Epoch 8/10
49

In [11]:
#For validation set
loss,acc=model.evaluate_generator(generator=validation_generator,steps=validation_generator.n//batch_size)
print('Loss for Validation Data :',loss)
print('Accuracy for Validation Data :',acc)

Loss for Validation Data : 0.3836861550807953
Accuracy for Validation Data : 0.9054545164108276


In [12]:
#For test set
scores=model.evaluate_generator(test_generator,steps=2520)
print('Loss for Test Data',loss)
print('Accuracy for Test Data',acc)

Loss for Test Data 0.3836861550807953
Accuracy for Test Data 0.9054545164108276


In [13]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=2520,
verbose=1)
print(pred.shape)

predicted_class_indices=np.argmax(pred,axis=1)
print(predicted_class_indices.shape)

Instructions for updating:
Please use Model.predict, which supports generators.
2520/2520 [==============================] - 20s 8ms/step
(2520, 62)
(2520,)


In [14]:
(test_generator.classes[:6])
print(train_generator.class_indices)

{'00000': 0, '00001': 1, '00002': 2, '00003': 3, '00004': 4, '00005': 5, '00006': 6, '00007': 7, '00008': 8, '00009': 9, '00010': 10, '00011': 11, '00012': 12, '00013': 13, '00014': 14, '00015': 15, '00016': 16, '00017': 17, '00018': 18, '00019': 19, '00020': 20, '00021': 21, '00022': 22, '00023': 23, '00024': 24, '00025': 25, '00026': 26, '00027': 27, '00028': 28, '00029': 29, '00030': 30, '00031': 31, '00032': 32, '00033': 33, '00034': 34, '00035': 35, '00036': 36, '00037': 37, '00038': 38, '00039': 39, '00040': 40, '00041': 41, '00042': 42, '00043': 43, '00044': 44, '00045': 45, '00046': 46, '00047': 47, '00048': 48, '00049': 49, '00050': 50, '00051': 51, '00052': 52, '00053': 53, '00054': 54, '00055': 55, '00056': 56, '00057': 57, '00058': 58, '00059': 59, '00060': 60, '00061': 61}


In [15]:
from sklearn.metrics import accuracy_score
print('Accuracy for test data is', (accuracy_score(predicted_class_indices,test_generator.classes)))

Accuracy for test data is 0.9273809523809524
